In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
train = pd.read_csv('data/train.csv')
train.head(5)

,ID,text,label
0,SUAVK39Z,I feel that it was better I dieAm happy,Depression
1,9JDAGUV3,Why do I get hallucinations?,Drugs
2,419WR1LQ,I am stresseed due to lack of financial suppor...,Depression
3,6UY7DX6Q,Why is life important?,Suicide
4,FYC0FTFB,How could I be helped to go through the depres...,Depression


In [3]:
x_value = train['text']
y_value = train['label'] #.replace({'Depression': 0, 'Drugs': 1, 'Suicide': 2, 'Alcohol': 4})
x_train, x_test, y_train, y_test = train_test_split(x_value, y_value, test_size=0.25)

In [4]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1',\
                        ngram_range=(1, 3), stop_words='english')
features = tfidf.fit_transform(x_train).toarray()
#labels = y_train
features.shape

(462, 84)

In [5]:
vect = CountVectorizer()
x_train_counts = vect.fit_transform(x_train)
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)
clf = MultinomialNB().fit(x_train_tfidf, y_train)

In [7]:
clf = LinearSVC()